In [ ]:
#Criação do container do container no Docker

docker run --name mysql_teste_sx -e MYSQL_ROOT_PASSWORD=123 -e MYSQL_DATABASE=bd_enem -p 3306:3306 -d mysql:latest

In [ ]:
#Imports de bibliotecas e conexão com o BD

#Imports de bibliotecas
import pandas as pd
from sqlalchemy import create_engine, Column, CHAR, VARCHAR, Boolean, Integer
from sqlalchemy.dialects.mysql import TINYINT, SMALLINT, BIGINT, DECIMAL
from sqlalchemy.ext.declarative import declarative_base
import numpy as np


#Parâmetros
usuario = 'root'
senha = '123'
host = 'localhost'
banco = 'bd_enem'

#Conexão
engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}/{banco}')

In [ ]:
#Leitura do arquivo csv e alteração dos tipos de dados do Dataframe

#Leitura do arquivo para um dataframe
df = pd.read_csv('DADOS/MICRODADOS_ENEM_2020.csv', sep=';', encoding='latin1', low_memory=False)

#Dicionário para definição dos tipos de dados para ajuste no dataframe
col_dict = {
    'NU_INSCRICAO': 'int64', 
    'NU_ANO': 'int16', 
    'TP_FAIXA_ETARIA': 'int8', 
    'TP_SEXO': 'string', 
    'TP_ESTADO_CIVIL': 'int8', 
    'TP_COR_RACA': 'int8', 
    'TP_NACIONALIDADE': 'int8', 
    'TP_ST_CONCLUSAO': 'int8', 
    'TP_ANO_CONCLUIU': 'int8', 
    'TP_ESCOLA': 'int8', 
    'TP_ENSINO': 'int8', 
    'IN_TREINEIRO': 'int8', 
    'CO_MUNICIPIO_ESC': 'int32', 
    'NO_MUNICIPIO_ESC': 'string', 
    'CO_UF_ESC': 'int8', 
    'SG_UF_ESC': 'string', 
    'TP_DEPENDENCIA_ADM_ESC': 'int8', 
    'TP_LOCALIZACAO_ESC': 'int8', 
    'TP_SIT_FUNC_ESC': 'int8', 
    'CO_MUNICIPIO_PROVA': 'int32', 
    'NO_MUNICIPIO_PROVA': 'string', 
    'CO_UF_PROVA': 'int8', 
    'SG_UF_PROVA': 'string', 
    'TP_PRESENCA_CN': 'int8', 
    'TP_PRESENCA_CH': 'int8', 
    'TP_PRESENCA_LC': 'int8', 
    'TP_PRESENCA_MT': 'int8', 
    'CO_PROVA_CN': 'int16', 
    'CO_PROVA_CH': 'int16', 
    'CO_PROVA_LC': 'int16', 
    'CO_PROVA_MT': 'int16', 
    'NU_NOTA_CN': 'float64', 
    'NU_NOTA_CH': 'float64', 
    'NU_NOTA_LC': 'float64', 
    'NU_NOTA_MT': 'float64', 
    'TX_RESPOSTAS_CN': 'string', 
    'TX_RESPOSTAS_CH': 'string', 
    'TX_RESPOSTAS_LC': 'string', 
    'TX_RESPOSTAS_MT': 'string', 
    'TP_LINGUA': 'bool', 
    'TX_GABARITO_CN': 'string', 
    'TX_GABARITO_CH': 'string', 
    'TX_GABARITO_LC': 'string', 
    'TX_GABARITO_MT': 'string', 
    'TP_STATUS_REDACAO': 'int8', 
    'NU_NOTA_COMP1': 'int8', 
    'NU_NOTA_COMP2': 'int8', 
    'NU_NOTA_COMP3': 'int8', 
    'NU_NOTA_COMP4': 'int8', 
    'NU_NOTA_COMP5': 'int8', 
    'NU_NOTA_REDACAO': 'int16', 
    'Q001': 'string', 
    'Q002': 'string', 
    'Q003': 'string', 
    'Q004': 'string', 
    'Q005': 'int8', 
    'Q006': 'string', 
    'Q007': 'string', 
    'Q008': 'string', 
    'Q009': 'string', 
    'Q010': 'string', 
    'Q011': 'string', 
    'Q012': 'string', 
    'Q013': 'string', 
    'Q014': 'string', 
    'Q015': 'string', 
    'Q016': 'string', 
    'Q017': 'string', 
    'Q018': 'string', 
    'Q019': 'string', 
    'Q020': 'string', 
    'Q021': 'string', 
    'Q022': 'string', 
    'Q023': 'string', 
    'Q024': 'string', 
    'Q025': 'string'
}

#Ajuste nos tipos de dados em cada coluna (para tipo inteiro, é necessário substituir vazios e infinitos por 0. De acordo com o descrito e analizado da base, essa ação não gera conflito com os dados já presentes na base)
for coluna, tipo in col_dict.items():
    if tipo.startswith('int') or tipo.startswith('float'):
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce').fillna(0).astype(tipo)

In [ ]:
#Criação das tabelas, tipagem de dados e importação no BD

#Declaração dos tipos de dados da tabela tb_dados_participante
base = declarative_base()
class base_full(base):
    __tablename__ = 'tb_dados_participante'
    NU_INSCRICAO = Column(BIGINT, primary_key=True)
    NU_ANO = Column(SMALLINT)
    TP_FAIXA_ETARIA = Column(TINYINT)
    TP_SEXO = Column(CHAR(1))
    TP_ESTADO_CIVIL = Column(TINYINT)
    TP_COR_RACA = Column(TINYINT)
    TP_NACIONALIDADE = Column(TINYINT)
    TP_ST_CONCLUSAO = Column(TINYINT)
    TP_ANO_CONCLUIU = Column(TINYINT)
    TP_ESCOLA = Column(TINYINT)
    TP_ENSINO = Column(TINYINT)
    IN_TREINEIRO = Column(Boolean)
    CO_MUNICIPIO_ESC = Column(Integer)
    CO_MUNICIPIO_PROVA = Column(Integer)

#Criação da tabela tb_dados_participante com tipos definidos
base.metadata.create_all(engine)

#Importação da tabela tb_dados_participante
df[['NU_INSCRICAO',
    'NU_ANO',
    'TP_FAIXA_ETARIA',
    'TP_SEXO',
    'TP_ESTADO_CIVIL',
    'TP_COR_RACA',
    'TP_NACIONALIDADE',
    'TP_ST_CONCLUSAO',
    'TP_ANO_CONCLUIU',
    'TP_ESCOLA',
    'TP_ENSINO',
    'IN_TREINEIRO',
    'CO_MUNICIPIO_ESC',
    'CO_MUNICIPIO_PROVA'
]].to_sql('tb_dados_participante', con=engine, if_exists='replace', index=False)

print('tb_dados_participante importada')
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#Declaração dos tipos de dados da tabela tb_dados_escolas
base = declarative_base()
class base_full(base):
    __tablename__ = 'tb_dados_escola'
    CO_MUNICIPIO_ESC = Column(Integer, primary_key=True)
    NO_MUNICIPIO_ESC = Column(VARCHAR(150))
    CO_UF_ESC = Column(TINYINT)
    SG_UF_ESC = Column(CHAR(2))
    TP_DEPENDENCIA_ADM_ESC = Column(TINYINT)
    TP_LOCALIZACAO_ESC = Column(TINYINT)
    TP_SIT_FUNC_ESC = Column(TINYINT)

#Criação da tabela tb_dados_escola com tipos definidos
base.metadata.create_all(engine)

#Importação da tabela tb_dados_escola
df[['CO_MUNICIPIO_ESC',
    'NO_MUNICIPIO_ESC',
    'CO_UF_ESC',
    'SG_UF_ESC',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC', 
]].drop_duplicates().to_sql('tb_dados_escola', con=engine, if_exists='replace', index=False)

print('tb_dados_escolas importada')
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#Declaração dos tipos de dados da tabela tb_dados_local_aplic
base = declarative_base()
class base_full(base):
    __tablename__ = 'tb_dados_local_aplic'
    CO_MUNICIPIO_PROVA = Column(Integer, primary_key=True)
    NO_MUNICIPIO_PROVA = Column(VARCHAR(150))
    CO_UF_PROVA = Column(TINYINT)
    SG_UF_PROVA = Column(CHAR(2))

#Criação da tabela tb_dados_local_aplic com tipos definidos
base.metadata.create_all(engine)

#Importação da tabela tb_dados_local_aplic
df[['CO_MUNICIPIO_PROVA', 
    'NO_MUNICIPIO_PROVA', 
    'CO_UF_PROVA', 
    'SG_UF_PROVA', 
]].drop_duplicates().to_sql('tb_dados_local_aplic', con=engine, if_exists='replace', index=False)

print('tb_dados_local_aplic importada')
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#Declaração dos tipos de dados da tabela tb_dados_provas
base = declarative_base()
class base_full(base):
    __tablename__ = 'tb_dados_provas'
    NU_INSCRICAO = Column(BIGINT, primary_key=True)
    TP_PRESENCA_CN = Column(TINYINT)
    TP_PRESENCA_CH = Column(TINYINT)
    TP_PRESENCA_LC = Column(TINYINT)
    TP_PRESENCA_MT = Column(TINYINT)
    CO_PROVA_CN = Column(SMALLINT)
    CO_PROVA_CH = Column(SMALLINT)
    CO_PROVA_LC = Column(SMALLINT)
    CO_PROVA_MT = Column(SMALLINT)
    NU_NOTA_CN = Column(DECIMAL(4,1))
    NU_NOTA_CH = Column(DECIMAL(4,1))
    NU_NOTA_LC = Column(DECIMAL(4,1))
    NU_NOTA_MT = Column(DECIMAL(4,1))
    TX_RESPOSTAS_CN = Column(CHAR(45))
    TX_RESPOSTAS_CH = Column(CHAR(45))
    TX_RESPOSTAS_LC = Column(CHAR(45))
    TX_RESPOSTAS_MT = Column(CHAR(45))
    TP_LINGUA = Column(TINYINT)
    TX_GABARITO_CN = Column(CHAR(45))
    TX_GABARITO_CH = Column(CHAR(45))
    TX_GABARITO_LC = Column(CHAR(45))
    TX_GABARITO_MT = Column(CHAR(45))
    TP_STATUS_REDACAO = Column(TINYINT)
    NU_NOTA_COMP1 = Column(TINYINT)
    NU_NOTA_COMP2 = Column(TINYINT)
    NU_NOTA_COMP3 = Column(TINYINT)
    NU_NOTA_COMP4 = Column(TINYINT)
    NU_NOTA_COMP5 = Column(TINYINT)
    NU_NOTA_REDACAO = Column(SMALLINT)

#Criação da tabela tb_dados_provas com tipos definidos
base.metadata.create_all(engine)

df[['NU_INSCRICAO',
    'TP_PRESENCA_CN', 
    'TP_PRESENCA_CH', 
    'TP_PRESENCA_LC', 
    'TP_PRESENCA_MT', 
    'CO_PROVA_CN', 
    'CO_PROVA_CH', 
    'CO_PROVA_LC', 
    'CO_PROVA_MT', 
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT', 
    'TX_RESPOSTAS_CN', 
    'TX_RESPOSTAS_CH', 
    'TX_RESPOSTAS_LC', 
    'TX_RESPOSTAS_MT', 
    'TP_LINGUA', 
    'TX_GABARITO_CN', 
    'TX_GABARITO_CH', 
    'TX_GABARITO_LC', 
    'TX_GABARITO_MT', 
    'TP_STATUS_REDACAO', 
    'NU_NOTA_COMP1', 
    'NU_NOTA_COMP2', 
    'NU_NOTA_COMP3', 
    'NU_NOTA_COMP4', 
    'NU_NOTA_COMP5', 
    'NU_NOTA_REDACAO', 
]].to_sql('tb_dados_provas', con=engine, if_exists='replace', index=False)

print('tb_dados_provas importada')
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#Declaração dos tipos de dados da tabela tb_dados_quest_soci
base = declarative_base()
class base_full(base):
    __tablename__ = 'tb_dados_quest_soci'
    NU_INSCRICAO = Column(BIGINT, primary_key=True)
    Q001 = Column(CHAR(1))
    Q002 = Column(CHAR(1))
    Q003 = Column(CHAR(1))
    Q004 = Column(CHAR(1))
    Q005 = Column(TINYINT)
    Q006 = Column(CHAR(1))
    Q007 = Column(CHAR(1))
    Q008 = Column(CHAR(1))
    Q009 = Column(CHAR(1))
    Q010 = Column(CHAR(1))
    Q011 = Column(CHAR(1))
    Q012 = Column(CHAR(1))
    Q013 = Column(CHAR(1))
    Q014 = Column(CHAR(1))
    Q015 = Column(CHAR(1))
    Q016 = Column(CHAR(1))
    Q017 = Column(CHAR(1))
    Q018 = Column(CHAR(1))
    Q019 = Column(CHAR(1))
    Q020 = Column(CHAR(1))
    Q021 = Column(CHAR(1))
    Q022 = Column(CHAR(1))
    Q023 = Column(CHAR(1))
    Q024 = Column(CHAR(1))
    Q025 = Column(CHAR(1))

#Criação da tabela tb_dados_quest_soci com tipos definidos
base.metadata.create_all(engine)

df[['NU_INSCRICAO',
    'Q001', 
    'Q002', 
    'Q003', 
    'Q004', 
    'Q005', 
    'Q006', 
    'Q007', 
    'Q008', 
    'Q009', 
    'Q010', 
    'Q011', 
    'Q012', 
    'Q013', 
    'Q014', 
    'Q015', 
    'Q016', 
    'Q017', 
    'Q018', 
    'Q019', 
    'Q020', 
    'Q021', 
    'Q022', 
    'Q023', 
    'Q024', 
    'Q025', 
]].to_sql('tb_dados_quest_soci', con=engine, if_exists='replace', index=False)

print('tb_dados_quest_soci importada')

In [ ]:
#Criação, carregamento e importação das Tabelas Dicionário no BD

#Criação da tabela tb_dicionario_colunas
dicionario_dec = declarative_base()
class dicionario(dicionario_dec):
    __tablename__ = 'tb_dicionario_colunas'
    tp_col = Column(VARCHAR(36))
    nm_col = Column(VARCHAR(22), primary_key=True)
    desc_col = Column(VARCHAR(277))
dicionario_dec.metadata.create_all(engine)

#Carregamento e importação da tabela tb_dicionario_colunas
df = pd.read_csv('aux_tabs/dicionario_cols.csv', sep=';', encoding='latin1', low_memory=False)
df.to_sql('tb_dicionario_colunas', con=engine, if_exists='replace', index=False)

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#Criação da tabela tb_dicionario_respostas
dicionario_dec = declarative_base()
class dicionario(dicionario_dec):
    __tablename__ = 'tb_dicionario_respostas'
    nm_col = Column(VARCHAR(22), primary_key=True)
    cat_resp = Column(VARCHAR(3), primary_key=True)
    desc_resp = Column(VARCHAR(437), primary_key=True)
dicionario_dec.metadata.create_all(engine)

#Carregamento e importação da tabela tb_dicionario_respostas
df = pd.read_csv('aux_tabs/dicionario_resp.csv', sep=';', encoding='latin1', low_memory=False)
df.to_sql('tb_dicionario_respostas', con=engine, if_exists='replace', index=False)

In [ ]:
# Querys para testes

# Este bloco de cõdigo serve para realizar testes de extrações das tabelas em SQL.
# Os resultados, quando satisfatõrios, são utilizados no Power Query dentro do Excel/Power BI utilizando a conexão do MySQL para gerar as tabelas de dados utilizadas nas análises e visões.

df = pd.read_sql("""

 
    


""",engine
)

df.to_csv("C:/Users/João/Documents/Teste Analista de Dados SX/extrac.csv")